In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/sentiment-models')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd

df_sub_xlnet = pd.read_csv("./input/sub_xlnet_op.csv")
df_sub_bert = pd.read_csv("./input/sub_bert_op.csv")
df_sub_roberta = pd.read_csv("./input/sub_roberta_op.csv")



In [ ]:
df_sub_xlnet["label_xlnet"] = df_sub_xlnet["label"]
df_sub_xlnet["label_bert"] = df_sub_bert["label"]
df_sub_xlnet["label_roberta"] = df_sub_roberta["label"]

df_sub_xlnet.drop(columns="label", axis=1, inplace=True)



In [ ]:
xlnet_bert_equals = df_sub_xlnet.loc[df_sub_xlnet['label_bert']==df_sub_xlnet['label_roberta']]

len(xlnet_bert_equals.loc[xlnet_bert_equals['label_xlnet']!=xlnet_bert_equals['label_bert']])

35

In [ ]:
labels = []

for index, row in df_sub_xlnet.iterrows():
    if(row['label_xlnet'] + row['label_bert'] + row['label_roberta'])>= 2:
      labels.append(1)
    else:
      labels.append(0)
    #print(row['label_xlnet'] + row['label_bert'] + row['label_roberta'])
df_sub_xlnet['label'] = labels

df_sub_xlnet.drop(columns="label_xlnet", axis=1, inplace=True)
df_sub_xlnet.drop(columns="label_bert", axis=1, inplace=True)
df_sub_xlnet.drop(columns="label_roberta", axis=1, inplace=True)

df_sub_xlnet

In [ ]:
# write predictions to a CSV file
df_sub_xlnet.to_csv("ensemble.csv", index=False)

In [ ]:
#################################

In [ ]:
test_data = pd.read_csv("./input/test_oJQbWVk.csv")
X_test = list(test_data["tweet"])

In [ ]:
# Load trained model
model_path = "xlnet-base-cased-finetuned-classification/checkpoint-3564"
model = XLNetForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_name = "xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained(model_name, do_lower_case=True)

# Define test trainer
trainer = Trainer(model)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Calculating probabilities

probs = []

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
for elem in X_test:
  probs.append(pipe(elem, return_all_scores=True))
#prediction = pipe("The text to predict", return_all_scores=True)


# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

sub = pd.DataFrame({'id':test_data['id'], 'label':y_pred})

# write predictions to a CSV file
sub.to_csv("sub_xlnet_op.csv", index=False)

In [ ]:
# Load trained model
model_path = "bert-base-uncased-finetuned-classification/checkpoint-3564"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define test trainer
trainer = Trainer(model)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

sub = pd.DataFrame({'id':test_data['id'], 'label':y_pred})

# write predictions to a CSV file
sub.to_csv("sub_bert_op.csv", index=False)

In [ ]:
# Load trained model
model_path = "bert-base-uncased-finetuned-classification/checkpoint-3564"
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_name = 'roberta-base' #'roberta-large' , 'roberta-large-mnli'

tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Define test trainer
trainer = Trainer(model)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

sub = pd.DataFrame({'id':test_data['id'], 'label':y_pred})

# write predictions to a CSV file
sub.to_csv("sub_roberta_op.csv", index=False)